In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import KFold
from scipy.stats import t

In [50]:
metadata_train = pd.read_csv("metadata.csv",header=0,index_col=0)
microbiome_train = pd.read_csv("microbiome.csv",header=0,index_col=0)
metabolome_train = pd.read_csv("serum_lipo.csv",header=0,index_col=0)
metadata_test = pd.read_csv("metadata_test.csv",header=0,index_col=0)
microbiome_test = pd.read_csv("microbiome_test.csv",header=0,index_col=0)
metabolome_test = pd.read_csv("serum_lipo_test.csv",header=0,index_col=0)

In [51]:
def clr_transform(df: pd.DataFrame, epsilon=1e-6) -> pd.DataFrame:
    clr_df = df.fillna(0.0).astype(float).values
    clr_df = np.log(clr_df + epsilon)
    clr_df = clr_df - clr_df.mean(axis=1, keepdims=True)
    return pd.DataFrame(clr_df, index=df.index, columns=df.columns)
def standartization_transform(train_df, test_df):
    scaler = StandardScaler().fit(train_df)
    train_scaled = pd.DataFrame(scaler.transform(train_df), index=train_df.index, columns=train_df.columns)
    test_scaled = pd.DataFrame(scaler.transform(test_df), index=test_df.index, columns=test_df.columns)
    return train_scaled, test_scaled

In [52]:
microbiome_train = clr_transform(microbiome_train)
metadata_train['Disease_status'] = metadata_train['PATGROUPFINAL_C'].apply(lambda x:0 if x == '8' else 1)
labels_train = metadata_train['Disease_status']
microbiome_test = clr_transform(microbiome_test)
metabolome_train, metabolome_test = standartization_transform(metabolome_train, metabolome_test)
microbiome_and_metabolome_train = pd.merge(metabolome_train, microbiome_train, left_index=True, right_index=True, how='inner')
microbiome_and_metabolome_test = pd.merge(metabolome_test, microbiome_test, left_index=True, right_index=True, how='inner')

In [53]:
# With mean between bags
def run_bagged_random_forest(train_data, train_labels, test_data):
    rand = np.random.default_rng()
    # labels: 0 = healthy, 1 = sick
    healthy_indices = np.where(train_labels == 0)[0]
    sick_indices    = np.where(train_labels == 1)[0]
    n_bags = 60
    target_sick_ratio = 0.1

    # Options for model to increase diversity between bags
    bag_size_low  = 150
    bag_size_high = 250
    max_features_pool   = ["sqrt", 0.2, 0.33, 0.5]
    max_depth_pool      = [4, 6, 8, 10, 12, None]
    min_samples_leaf_pool = [1, 2, 3, 4, 5, 6]

    probs_train_sum = np.zeros(len(train_labels), dtype=float)
    probs_test_sum  = np.zeros(len(test_data), dtype=float)
    probs_train_bags = []
    probs_test_bags  = []

    for b in range(n_bags):
        # Different bag size each time
        bag_size = int(rand.integers(bag_size_low, bag_size_high + 1))
        n_sick_bag    = max(1, int(round(target_sick_ratio * bag_size)))
        n_healthy_bag = max(1, bag_size - n_sick_bag)

        # Sampling to have 90% healthy 10% sick inside each bag
        sick_sample = rand.choice(
            sick_indices,
            size=n_sick_bag,
            replace=(n_sick_bag > len(sick_indices))
        )
        healthy_sample = rand.choice(
            healthy_indices,
            size=n_healthy_bag,
            replace=True
        )

        bag_indices = np.concatenate([healthy_sample, sick_sample])
        rand.shuffle(bag_indices)

        data_bag   = train_data.iloc[bag_indices]
        labels_bag = train_labels.iloc[bag_indices]

        # Choose model hyperparameters for current bag
        max_features_choice   = max_features_pool[int(rand.integers(0, len(max_features_pool)))]
        max_depth_choice      = max_depth_pool[int(rand.integers(0, len(max_depth_pool)))]
        min_samples_leaf_choice = min_samples_leaf_pool[int(rand.integers(0, len(min_samples_leaf_pool)))]

        model = RandomForestClassifier(
            n_estimators=200,
            class_weight="balanced_subsample",
            max_features=max_features_choice,
            max_depth=max_depth_choice,
            min_samples_leaf=min_samples_leaf_choice,
        )
        model.fit(data_bag, labels_bag)

        # Prediction probabilities per bag
        probs_train = model.predict_proba(train_data)[:, 1]
        probs_test  = model.predict_proba(test_data)[:, 1]

        probs_train_sum += probs_train
        probs_test_sum  += probs_test

        probs_train_bags.append(probs_train)
        probs_test_bags.append(probs_test)

    # Average probabilities across bags
    probs_train_mean = probs_train_sum / n_bags
    probs_test_mean  = probs_test_sum  / n_bags

    probs_train_bags = np.vstack(probs_train_bags)
    probs_test_bags  = np.vstack(probs_test_bags)
    
    return probs_train_mean, probs_test_mean, probs_train_bags, probs_test_bags

In [ ]:
def first_layer(train_data, train_labels, test_data):
    train_probs, test_probs, train_probs_bags, test_probs_bags = run_bagged_random_forest(train_data, train_labels, test_data)

    std_train = train_probs_bags.std(axis=0)
    std_test = test_probs_bags.std(axis=0)
    prob_lower, prob_upper = np.quantile(train_probs, [0.02, 0.6])
    std_threshold = np.quantile(std_train, 0.4)

    certain_test_samples_mask = ((test_probs <= prob_lower) | (test_probs >= prob_upper)) & (std_test <= std_threshold)
    uncertain_test_samples_mask = ~certain_test_samples_mask
    certain_indices = np.where(certain_test_samples_mask)[0]
    uncertain_indices  = np.where(uncertain_test_samples_mask)[0]
    test_data_uncertain = test_data.iloc[uncertain_indices]
    test_probs_certain = test_probs[certain_indices]
    return test_data_uncertain, test_probs_certain, test_probs[uncertain_indices], test_data.index[certain_indices], test_data.index[uncertain_indices]

In [55]:
def run_lgbm_with_reweighting(train_data, train_labels, test_data):
    """
    Fits a LightGBM classifier with sample weights to correct for class imbalance.
    The function assumes an initial distribution of 90% sick / 10% healthy in the training data
    and re-weights it to a target of 10% sick / 90% healthy.
    """
    target_ratio = 0.1
    sick_indices = train_labels == 1
    healthy_indices = train_labels == 0
    sick_num = sick_indices.sum()
    healthy_num = healthy_indices.sum()

    w_sick = 1.0
    w_healthy = (w_sick * sick_num * (1 - target_ratio)) / (target_ratio * healthy_num)
    
    # Assign weights per sample
    weights = np.ones_like(train_labels, dtype=float)
    weights[sick_indices] = w_sick
    weights[healthy_indices] = w_healthy

    lgbm = lgb.LGBMClassifier(objective='binary', verbose=-1)
    lgbm.fit(train_data, train_labels, sample_weight=weights)

    probs_test_uncertain = lgbm.predict_proba(test_data)[:, 1]
    return probs_test_uncertain

In [56]:
def second_layer(train_data, train_labels, test_data_uncertain):
    # Run the model on the uncertain data from the first layer
    probs_uncertain = run_lgbm_with_reweighting(train_data, train_labels, test_data_uncertain)
    return probs_uncertain

In [ ]:
def two_layer_model(train_data, train_labels, test_data):
    test_data_uncertain, test_probs_certain, test_probs_uncertain_layer1, test_certain_indices, test_uncertain_indices = first_layer(train_data, train_labels, test_data)
    test_probs_uncertain = second_layer(train_data, train_labels, test_data_uncertain)
    
    confidence_layer1 = np.abs(test_probs_uncertain_layer1 - 0.5)
    confidence_layer2 = np.abs(test_probs_uncertain - 0.5)
    final_uncertain = np.where(confidence_layer2 > confidence_layer1 , test_probs_uncertain, test_probs_uncertain_layer1)
    final_probs = pd.Series(index=test_data.index, dtype=float)
    final_probs.loc[test_certain_indices]  = test_probs_certain
    final_probs.loc[test_uncertain_indices] = final_uncertain
    df = final_probs.to_frame(name="Probability")
    df.index.name = "ID"
    return df

In [ ]:
results = two_layer_model(microbiome_and_metabolome_train, labels_train, microbiome_and_metabolome_test)
results.to_csv("output.csv")